In [ ]:
#installing necessary packages
%pip install litellm
%pip install -qU trulens_eval pydantic fastapi kaleido python-multipart uvicorn cohere openai tiktoken "llama-index"
%pip install transformers
%pip install sentence-transformers
%pip install pinecone-client
%pip install datasets
%pip install accelerate
%pip install einops
%pip install langchain
%pip install bitsandbytes

In [2]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [3]:
from torch import cuda, bfloat16
import os
import transformers
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
model_id = 'meta-llama/Llama-2-13b-chat-hf'

In [4]:
#If you get an error here during the first import from the `transformers` package, restart the kernel and try again.
from transformers import AutoTokenizer

# Get the Hugging Face token from the environment
hugging_face_token = os.environ.get('HF_AUTH')
model_name = 'meta-llama/Llama-2-13b-chat-hf'

# Load the correct tokenizer for Llama2 using your Hugging Face token
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hugging_face_token)


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
from transformers import LlamaTokenizer

# Get the Hugging Face token from the environment
hf_auth = os.environ.get('HF_AUTH')

# Load the tokenizer for Llama2 using our Hugging Face token
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf",use_auth_token=hf_auth)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1917: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [6]:
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
device = 'cuda:0' # make sure you are on gpu

embed_model = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [8]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=os.environ.get('HF_AUTH')
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

In [9]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bitsAndBites_config,
    device_map='auto',
    token=os.environ.get('HF_AUTH')
)

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [10]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    # we pass model parameters here too
    temperature=0.01,
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [11]:
import os
from pinecone import Pinecone, ServerlessSpec

# Get Pinecone API key and environment from environment variables
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Create an instance of the Pinecone class
pc = Pinecone(api_key=pinecone_api_key)

In [12]:
index_name = 'inlpt-project'
index = pc.Index(index_name)

In [13]:
from langchain.vectorstores import Pinecone

vectorstore = Pinecone(index, embed_model.embed_query, 'text')


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [14]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

In [15]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
chat_history = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_history.input_key="question"
chat_history.output_key="answer"

# generate llm pipeline with history chat
qa_conversation = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    memory=chat_history,
    return_source_documents=True
)


In [16]:
import pandas as pd

serachResults_df = pd.read_csv('Search results 20240102.csv')

In [ ]:
from ipywidgets import interact, widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Function to call rag_pipeline to generate answers from questions
def generate_answer_from_question(query):

    rag_result = qa_conversation(query)
    print(rag_result)
    answer = rag_result['answer']
    source_docs = rag_result['source_documents']
    celex_numbers = set([doc.metadata['CELEX number'] for doc in source_docs])

    answerWithSources = "\nAnswer:\n\n" + answer + "\n\nSources:\n\n"

    for celex in celex_numbers:
      title = serachResults_df.loc[serachResults_df['CELEX number'] == celex].iloc[0]['Title']
      celex = celex.replace("(", "%28").replace(")", "%29")
      url = "https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX%3A" + celex
      answerWithSources += title + "\n" + url + "\n"

    return answerWithSources



# Function to handle user input and display the generated answer
def handle_input(user_input):
    question = user_input
    print("\n\n"+ question)
    print('\nYour answer is being fetched...\n\n')
    answer = generate_answer_from_question({"question": question})
    print(answer)

# Create text input widget for user input
input_widget = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Input:',
    disabled=False
)

# Create button widget to trigger query processing
button_widget = widgets.Button(description="Submit")

# Define callback function when button is clicked
def on_button_clicked(b):
    user_input = input_widget.value
    handle_input(user_input)

button_widget.on_click(on_button_clicked)

# Display the widgets
display(input_widget)
display(button_widget)